In [0]:
# Impressions: 
# A million rows a day, where each list of impressions has 10 items -> 10 million rows a day
# item_id -> embeddings by transformer model (not handled by the pipeline) 
# need to give item_id(input feature) and is_order(target variable)

# Actions:
# clicks, add to carts and previous orders -> atleast one day before dt
# 150 million clicks and 15 million add to carts, and 2 million orders -> 7 days
# 21.43 million clicks and 2.14 million add to carts, and 0.29 million orders -> 1 days

# customer id in impressions data -> most recent 1000 actions in descending order -> going back at least 1 year
# for each day -> most recent sequence of actions for the corresponding customer id

# Required output parameters
# impressions: each integer is the embedding index for an item.
# actions: a list of 1000 actions, each integer corresponds to the embedding index of the interacted item.
# action_types: list of 1000 integers (1 for clicks, 2 for add to carts, 3 for previous orders, 0 for padding)



# Data Pipeline for Training Dataset

In [0]:
from pyspark.sql import functions as F, Window
from pyspark.sql import DataFrame
from pyspark.sql.functions import broadcast
from datetime import datetime

In [0]:
MAX_SEQ = 1000
PAD = 0
TYPE_CLICK = 1
TYPE_CART = 2
TYPE_ORDER = 3
TRAINING_WINDOW_DAYS = 14

In [0]:
# unify all action sources into one dataframe
def build_actions(clicks: DataFrame, add_to_carts: DataFrame, orders: DataFrame) -> DataFrame:
    try:        
        clicks_clean = clicks.select(
            F.col("customer_id"),
            F.col("item_id"),
            F.lit(TYPE_CLICK).alias("action_type"),
            F.col("click_time").alias("action_time")
        )

        carts_clean = add_to_carts.select(
            F.col("customer_id"),
            F.col("config_id").alias("item_id"),
            F.lit(TYPE_CART).alias("action_type"),
            F.col("occurred_at").alias("action_time")
        )

        orders_clean = orders.select(
            F.col("customer_id"),
            F.col("config_id").alias("item_id"),
            F.lit(TYPE_ORDER).alias("action_type"),
            F.col("order_date").cast("timestamp").alias("action_time")
        )

        actions = clicks_clean.unionByName(carts_clean).unionByName(orders_clean)
        return actions
    except Exception as e:
        print(f"Error building actions DataFrame: {e}")
        raise

In [0]:
# explode impressions
def explode_impressions(impressions: DataFrame) -> DataFrame:
    try:
        return impressions.withColumn("impression", F.explode("impressions")).select(
            "dt",
            "ranking_id",
            "customer_id",
            F.col("impression.item_id").alias("item_id"),
            F.col("impression.is_order").alias("is_order")
        )
    except Exception as e:
        print(f"Error exploding impressions: {e}")
        raise

In [0]:
def pad_or_truncate_array(col_name: str):
    try:
        return F.expr(f"slice(concat({col_name}, array_repeat({PAD}, {MAX_SEQ})), 1, {MAX_SEQ})")
    except Exception as e:
        print(f"Error padding or truncating array: {e}")
        raise

In [0]:
# get last 1000 actions per customer before 'dt' within 1 year
def get_customer_action_history(actions: DataFrame, impressions: DataFrame) -> DataFrame:
    try:
        # convert impression date to timestamp for filtering
        impressions = impressions.withColumn("impression_date", F.col("dt").cast(TimestampType()))

        # join and filter actions occurring strictly more than one day before the current impression
        # impressions per day are much smaller than actions; broadcasting avoids a shuffle-heavy join
        joined = broadcast(impressions.alias("imp")).join(
            actions.alias("act"),
            on="customer_id",
            how="inner"
        ).where(
            (F.col("act.action_time") < F.col("imp.impression_date"))
        )

        # rank actions per customer per impression date
        w = Window.partitionBy("imp.customer_id", "imp.impression_date").orderBy(F.col("act.action_time").desc())
        ranked = joined.withColumn("rn", F.row_number().over(w))

        # keep only last 1000 actions
        filtered = ranked.filter(F.col("rn") <= MAX_SEQ)

        # aggregate the collected action data, keep ordering before collect
        agg = filtered.groupBy("imp.dt","customer_id","imp.item_id","imp.is_order").agg(
            F.sort_array(F.collect_list(F.struct("rn", "act.item_id")), asc=True).alias("actions_struct"),
            F.sort_array(F.collect_list(F.struct("rn", "act.action_type")), asc=True).alias("action_types_struct")
        )

        # extract values back from struct into plain arrays, preserving alignment between actions and action_types
        agg = agg.withColumn("actions", F.expr("transform(actions_struct, x -> x.item_id)")) \
                .withColumn("action_types", F.expr("transform(action_types_struct, x -> x.action_type)")) \
                .drop("actions_struct", "action_types_struct")
        
        # apply padding and/or truncation to the collected arrays
        final = agg.withColumn("actions", pad_or_truncate_array("actions")) \
                .withColumn("action_types", pad_or_truncate_array("action_types"))

        return final
    except Exception as e:
        print(f"Error in customer action history: {e}")
        raise

In [0]:
# main pipeline
def build_training_dataset(impressions: DataFrame,
                           clicks: DataFrame,
                           add_to_carts: DataFrame,
                           orders: DataFrame,
                           end_date: str = None) -> DataFrame:
    try:
        if end_date is None:
            end_date = datetime.now().strftime("%Y-%m-%d")

        days_list = [(datetime.strptime(end_date, "%Y-%m-%d") - timedelta(days=i)).strftime("%Y-%m-%d")
                    for i in range(TRAINING_WINDOW_DAYS)]

        # impressions exploded for the last 14 days
        impressions_exploded = explode_impressions(impressions.where(F.col("dt").isin(days_list)))

        # build actions DataFrame from various sources
        actions = build_actions(clicks, add_to_carts, orders)

        # ensure data is scoped within a year prior to latest date in the window
        max_day = max(days_list)

        # restrict actions to only customers appearing in impressions window
        impression_customers = impressions_exploded.select("customer_id").distinct()
        actions = actions.join(impression_customers, "customer_id", "inner") \
                        .where(F.col("action_time") >= F.date_sub(F.to_date(F.lit(max_day)), 365))

        
        # get customer action history and compile training data
        training_df = get_customer_action_history(actions, impressions_exploded)
    
        # repartition output by dt for efficient downstream reads
        return training_df.repartition("dt")
    except Exception as e:
        print(f"Error in building training dataset: {e}")
        raise

## Generate Test Data

In [0]:
# add data generation notebook full path below

In [0]:
%run Test/data_generation

In [0]:
# generate test data using functions
impressions_df = generate_impressions()
clicks_df = generate_clicks()
add_to_cart_df = generate_add_to_cart()
previous_orders_df = generate_previous_orders()

## Build and Execute Training Dataset Pipeline

In [0]:
# execution
# replace test dataframes with production dataframes
try:
    training_df = build_training_dataset(impressions_df, clicks_df, add_to_cart_df, previous_orders_df)
except Exception as e:
    print(f"Error in pipeline execution: {e}")

In [0]:
training_df.display()

dt,customer_id,item_id,is_order,actions,action_types
2025-08-15,1016,259,false,"List(200, 204, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)","List(3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [0]:
# training_df.write.partitionBy("dt").mode("errorIfExists").format("parquet").save(output_path)